# Exploring results from DC power flow

In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd

import json
import os

from collections import defaultdict

In [2]:
data_dir = '~/research/ream-lab/CATS-CaliforniaTestSystem'

with open('pf_solution.json') as f:
    soln = json.load(f)
    print(soln['termination_status'])

LOCALLY_SOLVED


In [3]:
generators = pd.read_csv(os.path.join(data_dir, 'GIS', 'CATS_gens.csv'))
print(generators.describe)

<bound method NDFrame.describe of       PlantCode GenID   bus                              FuelType        Pg  \
0            34    1P   745            Conventional Hydroelectric  4.033363   
1           151     1  1804                     Petroleum Liquids  0.000000   
2           151     2  1804  Natural Gas Fired Combustion Turbine  0.000000   
3           161     1  1964            Conventional Hydroelectric  0.366669   
4           161     2  1964            Conventional Hydroelectric  0.366669   
...         ...   ...   ...                                   ...       ...   
3887          0     0  8862                 Synchronous Condenser  0.000000   
3888          0     0  8863                 Synchronous Condenser  0.000000   
3889          0     0  8864                 Synchronous Condenser  0.000000   
3890          0     0  8865                 Synchronous Condenser  0.000000   
3891          0     0  8867                 Synchronous Condenser  0.000000   

      Pmax  Pmin 

In [61]:
# Load California county boundaries shapefile
# Shapefile source https://gis.data.ca.gov/datasets/8713ced9b78a4abb97dc130a691a8695
county_shapes = gpd.read_file(os.path.join(data_dir, 'GIS', 'california_county_boundaries', 'cnty19_1.shp'))
county_shapes = county_shapes.to_crs("EPSG:4326")

# Create a GeoDataFrame from the generators DataFrame
generators_geo = gpd.GeoDataFrame(
    generators,
    geometry=gpd.points_from_xy(generators['Lon'], generators['Lat'], crs="EPSG:4326")
)

# Perform spatial join to get county information for each generator
joined_data = gpd.sjoin(generators_geo, county_shapes, how="left", predicate="within")

# Create a dictionary mapping PlantCode to county name
generator_to_county = dict(zip(joined_data['PlantCode'], joined_data['COUNTY_NAM']))

# Use the resulting dictionary
# Print top 5 dictionary entries for now
gen_to_county_slice = dict(list(generator_to_county.items())[0: 5])
print(str(gen_to_county_slice))


{34: 'Placer', 151: 'Stanislaus', 161: 'Stanislaus', 162: 'Stanislaus', 180: 'Shasta'}


Now we have a dictionary of generators to county. Next we iterate through the solution to assign each generator to a county, which allows us to aggregate generation mix at the county level.

In [108]:
# Create a dict mapping county to generation mix
county_mix = defaultdict(list)

# Process generators table to get unique PlantCodes
plant_codes = list(set(generators_geo.PlantCode) - {0})

for g in plant_codes:
    if str(g) in soln['solution']['gen']:
        county = generator_to_county[g]
        county_mix[county].append((g, soln['solution']['gen'][str(g)]["pg"]))
    else:
        print("Generator %i missing from solution", g)

county_mix

Generator %i missing from solution 61442
Generator %i missing from solution 61443
Generator %i missing from solution 61444
Generator %i missing from solution 61445
Generator %i missing from solution 57359
Generator %i missing from solution 57360
Generator %i missing from solution 57361
Generator %i missing from solution 57364
Generator %i missing from solution 61463
Generator %i missing from solution 57378
Generator %i missing from solution 57394
Generator %i missing from solution 61501
Generator %i missing from solution 61502
Generator %i missing from solution 61503
Generator %i missing from solution 57422
Generator %i missing from solution 57439
Generator %i missing from solution 57441
Generator %i missing from solution 57455
Generator %i missing from solution 57459
Generator %i missing from solution 57460
Generator %i missing from solution 57475
Generator %i missing from solution 57482
Generator %i missing from solution 57483
Generator %i missing from solution 57484
Generator %i mis

defaultdict(list,
            {'Placer': [(34, 0.3052635343410021),
              (214, 0.4265952363126877),
              (235, -9.678005300484403e-09),
              (237, 1.0036538028976498),
              (241, 0.3068434663244677),
              (292, 0.1806121258374966),
              (424, 0.20131765242148753),
              (425, 1.1870000116211468),
              (426, 0.10661263806961903),
              (427, -9.95322784146443e-09),
              (632, 0.05364106864010107)],
             'Stanislaus': [(151, 0.29426438779140407),
              (161, -9.516592395332016e-09),
              (162, 0.059000008981459705)],
             'Shasta': [(180, 0.06997553388403024),
              (227, 0.531169592926797),
              (229, 1.17393245489778),
              (243, 1.3031915293173446),
              (244, 1.2120534250337538),
              (249, 0.33856211717938123),
              (253, 0.10000000810759982),
              (265, 0.3619039071227875),
              (267, 0.380490

In [154]:
def gen_share_for_list(gen_list):
    grouped_gen_list = defaultdict(float)

    for gen, pg in gen_list:
        fuel_type = generators_geo.loc[generators_geo.PlantCode == gen].FuelType.iloc[0]
        grouped_gen_list[fuel_type] += pg

    total = sum(grouped_gen_list.values())

    return {fuel: total_gen / total for fuel, total_gen in grouped_gen_list.items()}

county_gen_share = {county: gen_share_for_list(gen_list) for county, gen_list in county_mix.items()}

# Print or use county_gen_share
print(county_gen_share)


{'Placer': {'Conventional Hydroelectric': 1.0}, 'Stanislaus': {'Petroleum Liquids': 0.8329862799839303, 'Conventional Hydroelectric': 0.16701372001606962}, 'Shasta': {'Conventional Hydroelectric': 1.0}, 'Calaveras': {'Conventional Hydroelectric': 1.0}, 'Plumas': {'Conventional Hydroelectric': 1.0}, 'El Dorado': {'Conventional Hydroelectric': 1.0}, 'Amador': {'Conventional Hydroelectric': 1.0}, 'Fresno': {'Conventional Hydroelectric': 1.0}, 'Humboldt': {'Natural Gas Internal Combustion Engine': 1.0}, 'Tehama': {'Conventional Hydroelectric': 1.0}, 'Merced': {'Conventional Hydroelectric': 1.000000032164672, 'Hydroelectric Pumped Storage': -3.216467213312131e-08}, 'Monterey': {'Natural Gas Fired Combined Cycle': 1.0}, 'Tuolumne': {'Conventional Hydroelectric': 1.0}, 'Mendocino': {'Conventional Hydroelectric': 1.0}, 'Nevada': {'Conventional Hydroelectric': 1.0}, 'Tulare': {'Conventional Hydroelectric': 1.0}, 'Siskiyou': {'Conventional Hydroelectric': 1.0}, 'Imperial': {'Conventional Hydroel